# Setup Environment
Configure the Google Colab environment for working with geospatial data.

In [ ]:
# Setup Environment by install fortis
!pip install https://github.com/TroySchmidt/fortis/releases/download/0.1.0/fortis_data-0.1.0-py3-none-any.whl
!pip install https://github.com/TroySchmidt/fortis/releases/download/0.1.0/fortis_engine-0.1.0-py3-none-any.whl

# Install required packages if needed
!pip install requests tqdm

# Download the NSI data
Need to download the NSI data from the site.

In [ ]:
import requests
import os
import zipfile
from tqdm.notebook import tqdm

def download_and_extract_zip(url, target_dir="."):
    """
    Download a zip file from a URL and extract its contents.

    Args:
        url (str): URL of the zip file to download
        target_dir (str): Directory to extract the contents to
    """
    # Create target directory if it doesn't exist
    os.makedirs(target_dir, exist_ok=True)

    # Get the filename from URL
    filename = url.split('/')[-1]
    filepath = os.path.join(target_dir, filename)

    # Download the file with progress bar
    print(f"Downloading {filename}...")
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))

    with open(filepath, 'wb') as f, tqdm(
            desc=filename,
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
        for data in response.iter_content(chunk_size=1024):
            size = f.write(data)
            bar.update(size)

    # Extract the zip file
    print(f"Extracting {filename}...")
    with zipfile.ZipFile(filepath, 'r') as zip_ref:
        zip_ref.extractall(target_dir)

    print(f"Download and extraction complete. Files extracted to {target_dir}")

    # Return the path to the extracted gpkg file (assumes there's only one gpkg file)
    extracted_files = [f for f in os.listdir(target_dir) if f.endswith('.gpkg')]
    if extracted_files:
        return os.path.join(target_dir, extracted_files[0])
    return None

# Download and extract the NSI data
nsi_url = "https://nsi.sec.usace.army.mil/downloads/nsi_2022/nsi_2022_15.gpkg.zip"
gpkg_path = download_and_extract_zip(nsi_url, "nsi_data")

print(f"GeoPackage path: {gpkg_path}")

Download the raster.

In [ ]:
import requests
from tqdm.notebook import tqdm  # Import tqdm for Jupyter Notebook

def download_file_with_progress(url, filename):
    """Downloads a file from a URL with a progress bar."""
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

        total_size = int(response.headers.get("content-length", 0))
        block_size = 8192  # Chunk size

        with open(filename, "wb") as f:
            with tqdm(total=total_size, unit="B", unit_scale=True, desc=filename) as pbar:
                for chunk in response.iter_content(chunk_size=block_size):
                    f.write(chunk)
                    pbar.update(len(chunk))

        print(f"Downloaded {filename} successfully.")
        return True

    except requests.exceptions.RequestException as e:
        print(f"Error downloading {filename}: {e}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False


# Download the file with progress
download_file_with_progress("https://github.com/TroySchmidt/fortis/raw/refs/heads/main/examples/Oahu_10_withReef.tif", "Oahu_10_withReef.tif")
download_file_with_progress("https://github.com/TroySchmidt/fortis/raw/refs/heads/main/examples/Oahu_10_withReef.tfw", "Oahu_10_withReef.tfw")

# Run the Analysis
Implement the NsiBuildings class that handles loading and processing NSI building data from GeoPackage files.

In [ ]:
import os
from fortis.engine.analyses.hazus_flood import HazusFloodAnalysis
from fortis.engine.models.flood_depth_grid import FloodDepthGrid
from fortis.engine.vulnerability.default_flood import DefaultFloodFunction
from fortis.engine.models.nsi_buildings import NsiBuildings

# Define file paths (adjust these paths as necessary)
buildings_gpkg = "nsi_data/nsi_2022_15.gpkg"
tif_file = "Oahu_10_withReef.tif"

# Load NSI buildings data from GPKG
buildings = NsiBuildings(buildings_gpkg, "nsi")

# Read the depth grid from the TIFF file
depth_grid = FloodDepthGrid(tif_file)

# Create an instance of the default flood function
flood_function = DefaultFloodFunction(buildings, flood_type="R")

# Create the Hazus flood analyzer instance
analyzer = HazusFloodAnalysis(
    buildings=buildings,
    vulnerability_func=flood_function,
    depth_grid=depth_grid,
)

# Calculate losses using the analysis
analyzer.calculate_losses()

# Save the results to multiple formats
results_base = "nsi_flood_losses"

# 1. Save as CSV
csv_path = f"{results_base}.csv"
buildings.gdf.to_csv(csv_path, index=False)

# 2. Save as GeoPackage
gpkg_path = f"{results_base}.gpkg"
buildings.gdf.to_file(gpkg_path, driver="GPKG")

print(f"NSI Flood analysis of {len(buildings.gdf):,} buildings complete.")
print(f"Results saved to:")
print(f"  - CSV: {csv_path}")
print(f"  - GeoPackage: {gpkg_path}")

: 

# Visualizations
Create visualizations of the results.

In [ ]:
import matplotlib.pyplot as plt

# Access the GeoDataFrame
gdf = buildings.gdf

# Filter out rows with None values in bldg_loss
filtered_gdf = gdf[gdf['bldg_loss'] > 0]

# Create the scatter plot with a different colormap
plt.figure(figsize=(10, 6))
scatter = plt.scatter(filtered_gdf.geometry.x, filtered_gdf.geometry.y, c=filtered_gdf['bldg_loss'], cmap='inferno', s=10,
                    vmin=filtered_gdf['bldg_loss'].min(), vmax=filtered_gdf['bldg_loss'].max())

# Customize the colorbar
colorbar = plt.colorbar(scatter, label='Building Loss', extend='both')  # Add extend='both'
plt.title('Building Loss by Location (Excluding None Values)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

# Create the histogram
plt.figure(figsize=(10, 6))
plt.hist(filtered_gdf['bldg_loss'], bins=50, edgecolor='black')  # Adjust bins as needed
plt.title('Distribution of Building Loss Values')
plt.xlabel('Building Loss')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()